# FUTURE SALES PREDICTION PROJECT

## The dataset contains sales information from Jan 2013 to Oct 2015. The task is to forecast the total amount of products sold in every shop for the test set. A warning of change in item number and shop id shoudl be taken into consideration.

### File despcription from Kaggle


**sales_train.csv    :** The training set. Daily historical data from January 2013 to October 2015.\
**test.csv           :** The test set. Forecast the sales for these shops and products for November 2015.\
**items.csv          :** Supplemental information about the items/products.\
**item_categories.csv:** Supplemental information about the items categories.\
**shops.csv          :** Supplemental information about the shops.

### Data fields from Kaggle

**ID** an Id that represents a (Shop, Item) tuple within the test set.\
**shop_id** unique identifier of a shop.\
**item_id** unique identifier of a product.\
**item_category_id** unique identifier of item category.\
**item_cnt_day** number of products sold. You are predicting a monthly amount of this measure.\
**item_price** current price of an item.\
**date** date in format dd/mm/yyyy.\
**date_block_num** a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33.\
**item_name** name of item.\
**shop_name** name of shop.\
**item_category_name** name of item category.

### Importing the Dataset and libraries

In [79]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



### Converting csv to Data frame 

In [80]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cat = pd.read_csv ('../input/competitive-data-science-predict-future-sales/item_categories.csv')
trainset1 = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
testset= pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

### To understand the basic structre of the tables

In [81]:
trainset1.info()

In [82]:
items.info()

In [83]:
item_cat.info()

In [84]:
shops.info()

### To check the number of records and features in trainset


In [85]:
trainset1.shape

### To explore each data set

In [86]:
display(trainset1, item_cat, items, shops)

### To merge the data sets; shops, items, item_cat, and trainset1 to get a better understanding of features 

In [87]:
trainset2=pd.merge(items, trainset1)

In [88]:
trainset3=pd.merge(item_cat, trainset2)

In [89]:
train=pd.merge(shops, trainset3)

### To understand the basic structurce of the new train data set

In [90]:
train.info()

In [91]:
train.shape

##### The number of rows of train and trainset1 are same 2935849, which shows that merge was successful.

In [92]:
train.describe()

##### In the statistical summary, notice that an item has a negative price which represents a mistake, also notice that item_cnt_day has a negative value which shows that there wasn't any sale that day. It shows -1 porbably because the products were returned. These issues will be fixed later during exploration.

### To check for missing values

In [93]:
train.isnull().sum()

##### The data set does not contain any missing values.

### To check the dimensions in categorical features

In [94]:
train['shop_name'].unique()

In [95]:
train['shop_id'].unique()

In [96]:
train['item_category_name'].unique()

In [97]:
train['item_name'].unique()

In [98]:
train['item_id'].unique()

In [99]:
train['shop_name'].nunique()

In [100]:
train['shop_id'].nunique()

##### This shows that each shop_name has an assigned shop_id

In [101]:
train['item_category_name'].nunique()

In [102]:
train['item_name'].nunique()

In [103]:
train['item_id'].nunique()

##### This shows that each item_name has an assigned item_id

### To copy the train data set

In [104]:
train_set=train.copy()

### To explore the dataset (EDA)

In [105]:
corrmat=train_set.corr()
corrmat

In [106]:
plt.figure(figsize=(15, 15))
sns.set(font_scale=2)
sns.heatmap(train_set.corr(), annot=True, cmap="Blues", fmt= ".2g")

### Imputing the negative item price.

In [107]:
negative_price=train_set[train_set["item_price"] < 0]
negative_price


##### There is only one record with negative price. Lets find the price of the item sold in the same shop during the same month, so we can imupte the value.


In [108]:
price_2973=train_set[(train_set["shop_id"]== 32) & (train_set["item_category_name"]=="Игры - PS3") & (train_set["item_id"]== 2973 ) & (train_set["date_block_num"]== 4)]
price_2973

##### Lets replace the price of the item with the average of two same items sold in the same month in the same shop.

In [109]:
price_mean=price_2973[(price_2973.shop_id==32)&(price_2973.item_id==2973)&(price_2973.date_block_num==4)&(price_2973.item_price>0)]
price_mean.item_price.mean()

In [110]:
train_set.at[1644072,'item_price']= 1849

### Imputing the negative value for item_cnt_day. 

In [111]:
negative_item_cnt_day=train_set[train_set["item_cnt_day"] < 0]
negative_item_cnt_day


##### There are 7356 rows which have the value of -1 for item_cnt_day, since our task is to predict the sales for the next month our focus should be on sales and not on the return. We will keep these rows in the data set because it can play a siginifacant factor to correctly predict and understand sales but we will replace the -1 with 0.

In [112]:
cond = train_set['item_cnt_day'] < 0
check= train_set.loc[cond,'item_cnt_day'] = 0
check


##### This shows we have successfully replaced all the -1's with 0's.

### To check for outliers

#### Outliers in item_price

In [113]:
plt.figure(figsize=(10, 15))
train_set.boxplot(['item_price'])
plt.title('Outliers in item_price', fontsize = 40)


##### There are several in item_price outliers and these values should be explored further.

In [114]:
# To find the highest possible price within 3 standard deviation
print("Maximum possible price",train_set['item_price'].mean() + 3*train_set['item_price'].std())
print("Lowest possible price",train_set['item_price'].mean() - 3*train_set['item_price'].std())


In [115]:
# To check the highest and lowest price in our dataset
hp=max(train_set.item_price)
print("Highest price :", hp)
lp=min(train_set.item_price)
print("Lowest price: ", lp)

In [116]:
# Since our lowest selling price is 0.07 we will check the outliers on the higher side
highest_price=train_set[train_set["item_price"] > 6080]
highest_price

##### There are 28701 records with a price of higher than $6080, and it is not suggested to drop all these values, especially considering that the fact that the data pertains to many different products. It is highly possible that the shops have a wide range of products with different prices so we will only eliminate the extreme outliers for item_price.

In [117]:
# To drop the outlier in item_price
train_set = train_set[train_set['item_price'] < 100000]

#### Outlier in item_cnt_day

In [118]:
plt.figure(figsize=(10, 15))
train_set.boxplot(['item_cnt_day'])
plt.title('Outliers in item_cnt_day', fontsize = 40)


##### There are several outliers in item_cnt_day and should be explored further

In [119]:
# To find the highest and lowest possible nuber of products sold within 3 standard deviation
print("Maximum possible number of products sold",train_set['item_cnt_day'].mean() + 3*train_set['item_cnt_day'].std())
print("Lowest possible number of products sold",train_set['item_cnt_day'].mean() - 3*train_set['item_cnt_day'].std())


In [120]:
# To check for maximum and lowest number of products sold
hps=max(train_set.item_cnt_day)
print("Highest number of products sold :", hps)
lps=min(train_set.item_cnt_day)
print("Lowest number of products sold: ", lps)

In [121]:
# Since our lowest sold quantity is zero we will check the outliers on the higher side
highest_price=train_set[train_set["item_cnt_day"] > 9]
highest_price

##### There are 11936 records on which the items sold is greater than 9. To reduce the impact of outliers and still be able to capture the impact of these sales we will only eliminate the extreme outliers for item_cnt_day.

In [122]:
# To drop the extreme outliers in item_cnt_day
train_set = train_set[train_set['item_cnt_day'] < 1000]

### Creating new features and extracting date

In [123]:
train_set['Sales'] = (train_set["item_price"] * train_set["item_cnt_day"])

In [ ]:
train_set['date'] = pd.to_datetime(train_set['date'])
days = []
months = []
years = []

for day in train_set['date']:
    days.append(day.day)
for month in train_set['date']:
    months.append(month.month)    
for year in train_set['date']:
    years.append(year.year)

### To find the year with highest sales

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(years)
plt.title('Sales per Year', fontsize = 40)
plt.xlabel('Year', fontsize = 25)
plt.ylabel('Sales', fontsize = 25)
plt.show()

##### This shows 2013 has the highest total sales.

### To find the highest selling product

In [ ]:
plt.figure(figsize=(40, 50))
x= train_set['item_category_id']
y= train_set['Sales']
sns.barplot(x, y)
plt.title('Highest Selling Product', fontsize = 60)
plt.xlabel('Item categories', fontsize = 30)
plt.ylabel('Sales', fontsize = 30)
plt.show()

##### The highest selling item category is 12, lets further explore the item_category_id 12

In [ ]:
highest_selling_itemc= train_set.loc[train_set['item_category_id'] == 12]
highest_selling_itemc

##### The highest selling item category is Игровые консоли - PS4

### To find the highest shop with highest total sales

In [ ]:
plt.figure(figsize=(40, 50))
x= train_set['shop_id']
y= train_set['Sales']
sns.barplot(x, y)
plt.title('Highest Selling Shop', fontsize = 60)
plt.xlabel('Shop_id', fontsize = 30)
plt.ylabel('Sales', fontsize = 30)
plt.show()

##### The highest selling shop has the id of 9

In [ ]:
highest_selling_shop= train_set.loc[train_set['shop_id'] == 9]
highest_selling_shop

##### The highest selling shop is Выездная Торговля	

### To find the month which contirbutes highest towards the total sales

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(months)
plt.title('Sales per Year', fontsize = 40)
plt.xlabel('Year', fontsize = 25)
plt.ylabel('Sales', fontsize = 25)
plt.show()

### To find the month from Jan 2013- Oct 2013 which had the highest sales

In [ ]:
sales_by_month = train_set.groupby(['date_block_num'])[["Sales"]].sum()
plt.figure(figsize=(30,30))
sales_by_month.plot()
plt.title('Month with highest Sales in the dataset', fontsize = 10)
plt.xlabel('Months', fontsize = 10)
plt.ylabel('Sales', fontsize = 10)

In [ ]:
sales_max = train_set.groupby(['date_block_num'])['Sales'].sum()
sales_max.max()

In [ ]:
print(sales_max==232615420.90999845)

##### The highest sale was in the 23 date block. Lets explore the date block 23.

In [ ]:
highest_sales_month = train_set[train_set["date_block_num"]==23]
highest_sales_month


##### This shows that the highest sales were in the month of December 2014

### To find the store which recorded highest sales in amonth from Jan 2013-Oct2015

In [ ]:
highest_store_sales = train_set.groupby(['date_block_num', 'shop_name'])['Sales'].sum()

highest_store_sales.max()

In [ ]:
highest_store_sales=pd.DataFrame(highest_store_sales)
highest_store_sales=highest_store_sales.loc[highest_store_sales['Sales'] == 15730394.000000501]
highest_store_sales

##### The store with the highest sales in a month from Jan 2013-Oct 2015 is Москва ТРК "Атриум"

### To find the item category which recorded highest sale in month from Jan 2013-Oct 2015

In [ ]:
highest_item_sales = train_set.groupby(['date_block_num', 'item_category_name'])['Sales'].sum()

highest_item_sales.max()

In [ ]:
highest_item_sales=pd.DataFrame(highest_item_sales)
highest_item_sales=highest_item_sales.loc[highest_item_sales['Sales'] == 46487721.9999976]
highest_item_sales

##### The item selling highest in a given month is Игровые консоли - PS4

### To aggregate train data set to show monthly data to bring it in line with test data

In [ ]:
train_set = train_set[["date_block_num", "shop_id", "item_id", "item_price", "item_cnt_day", "shop_name", "Sales", "item_name"]].groupby(
    ["date_block_num", "shop_id", "item_id"]).agg(
    {"item_cnt_day": "sum", "Sales" : "sum"}).reset_index()
train_set.rename(columns={"item_cnt_day": "item_cnt_month"}, inplace=True)

### To merge the test data with other datasets.

In [ ]:
test1=pd.merge(items, testset)
test2=pd.merge(item_cat, test1)
test = pd.merge(shops,test2)
test

### To drop the insignificant column ID because the same information is captured in shopi_id and item_id

In [ ]:
test=test.drop("ID",1)

### To explore the basic structure of test data set. 

In [ ]:
test.shape

### To align train data set with test. It is mentioned that we need to predict the sales of products and shops present in the test data set, therefore we need to check the dimenions of shop_id and item_id between train and test data set.

In [ ]:
train_set_u = train_set['item_id'].nunique()
test_u = test['item_id'].nunique()
train_set_s = train_set['shop_id'].nunique()
test_s = test['shop_id'].nunique()
print("Total unique item_ids in train_set dataset: ", train_set_u)
print("Total unique item_ids in test dataset: ", test_u )

print("Total unique shop_ids train_set dataset: ", train_set_s)
print("Total unique shop_ids in test dataset: ", test_s)

### To remove additional products and shops from the train data set.

In [ ]:
train_set = train_set[train_set['shop_id'].isin(test['shop_id'].unique())]
train_set = train_set[train_set['item_id'].isin(test['item_id'].unique())]
 

### To verify again

In [ ]:
train_set_u = train_set['item_id'].nunique()
test_u = test['item_id'].nunique()
train_set_s = train_set['shop_id'].nunique()
test_s = test['shop_id'].nunique()
print("Total unique item_ids in train_set dataset: ", train_set_u)
print("Total unique item_ids in test dataset: ", test_u )

print("Total unique shop_ids train_set dataset: ", train_set_s)
print("Total unique shop_ids in test dataset: ", test_s)

##### Notice now we have some item_ids not present in the train data set which would make it difficult to predict the sales because we don't have the information for these item_ids in train data set.

#### Create a reduced version of test and train to compare with full data set features

In [ ]:
test.shape

In [ ]:
test["item_cnt_month"]=" "

In [ ]:
test.info()
train_set.info()

In [ ]:
# To drop some insignificant columns
train_set=train_set.drop("Sales", 1)
train_set=train_set.drop("date_block_num", 1)
test= test.drop("item_category_id", 1)
test= test.drop("item_category_name", 1)
test= test.drop("item_name", 1)
test= test.drop("shop_name", 1)


 ### LGBMRegressoor Model 

In [ ]:
train_x = train_set.drop(['item_cnt_month'], axis=1)
train_y = train_set['item_cnt_month']
test_x = test.drop(['item_cnt_month'], axis=1)
model_lgb = LGBMRegressor(colsample_bytree=0.9, learning_rate=0.03, max_depth=6,
              min_child_weight=1, min_split_gain=0.0222415, n_estimators=100,
              num_leaves=32, reg_alpha=0.04, reg_lambda=0.073,
              subsample=0.9)
model_lgb.fit(train_x, train_y)



### Prediction

In [ ]:
sales_prediction = model_lgb.predict(test_x)
sales_prediction

### Submission

In [ ]:
sales_predict_submission = pd.DataFrame({'ID':test_x.index,'item_cnt_month':sales_prediction})

In [ ]:
sales_predict_submission

In [ ]:
sales_predict_submission.to_csv('./salespredictsubmission.csv', index= False)